# Jupyter Class Helper
---
These files are used to configure and organize the website's contents.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
# Always run this before any of the following cells
import pandas as pd
import numpy as np
import csv
import logging
import subprocess
import yaml
import builder as bd
from pathlib import Path
import gdown
base_path=Path('..')
config_path = base_path / 'config'
cf=bd.load_yaml_file(config_path / 'config.yml')
excel_file= config_path / cf['excel_file']
class_path= base_path / cf['class']
content_path = class_path / 'content'
cf


ordereddict([('excel_file', 'class.xlsx'), ('google_fileid', '1E0Y0lC-q6uYh6So-VEVT60k7_KJFn_xH'), ('class', 'site'), ('syllabus_message', 'You should always look to this website for the most recent updates.\\n\\nAn initial version of the syllabus can be found below. \\n\\n'), ('bc_class_header', '\\nThe following materials will be covered during the class session.\\n'), ('in_class_header', '\\nPlease review following materials before the class session.\\n'), ('assignments_header', '\\nAll assignments should be submitted to the LMS at 11:59 PM on the date indicated.\\n'), ('schedule_header', '\nPlease check here for the latest course schedule of activities.\n'), ('website_url', 'introml.analyticsdojo.com')])

In [3]:
url = 'https://drive.google.com/uc?id='+cf['google_fileid']
gdown.download(url, str(config_path / cf['excel_file']), quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1E0Y0lC-q6uYh6So-VEVT60k7_KJFn_xH
To: /Users/jasonkuruzovich/GitHub/fall-2021-introml-autobuild/config/class.xlsx
100%|██████████| 56.1k/56.1k [00:00<00:00, 4.55MB/s]


'../config/class.xlsx'

In [4]:
# These load configuration from the excel files 
config = bd.load_yaml_file(class_path / '_config.yml') # Load the file.
toc = bd.load_yaml_file(class_path / '_toc.yml') # Load the file.
config_xl= pd.read_excel(excel_file, sheet_name = '_config_yml', header=None, index_col=None)
schedule= pd.read_excel(excel_file, sheet_name = 'Schedule',  index_col=None)
content={}
content['Before Class']= pd.read_excel(excel_file, sheet_name = 'Before',  index_col=None)
content['In Class']= pd.read_excel(excel_file, sheet_name = 'During',  index_col=None)
#content['Assignment']= pd.read_excel(excel_file, sheet_name = 'Assignments',  index_col=None)


In [5]:
#Create the syllabus link.
#The second value of the index postion of the syllabus on the before class content.
#bd.create_syllabus(content['Before Class'],0,cf['syllabus_message'],content_path / 'syllabus.md', config['repository']['url'])

In [6]:
#Fix in case individual tries to publish where session is NA. This isn't allowed. 
schedule.loc[schedule['Session'].isna(),'Publish']=0. 

In [7]:
#Generate Links from the schedule to the sessions and within the other tables. 
schedule.loc[schedule['Publish']==1,'Location']=schedule.loc[schedule['Publish']==1,'Session'].apply(lambda x: '../sessions/session'+str(int(x)))
schedule.loc[schedule['Publish']==1,'Type']='Markdown'
schedule=bd.link_generator(schedule, 'Summary',config['repository']['url'],'Link')
#content['Assignment']=bd.link_generator(content['Assignment'], 'Assignment',config['repository']['url'],'Starter')
content['Before Class']=bd.link_generator(content['Before Class'], 'Content',config['repository']['url'],'Link')
content['In Class']=bd.link_generator(content['In Class'], 'Content',config['repository']['url'],'Link')

In [8]:
#Get the in class activities and prepare and output a markdown file. 
schedule_ic=schedule.merge(content['In Class'], left_on='Session', right_on='Session', how='left')
schedule_ic= schedule_ic.loc[schedule_ic['Content'].notnull(),['Week', 'Session', 'Date', 'Content']]
schedule_ic=bd.pandas_to_md(schedule_ic, content_path / 'in_class.md', 'In Class', \
        include = ['Week', 'Session', 'Date', 'Content'], header=cf['in_class_header'])

Converting datetime to 
Outputting file: ../site/content/in_class.md


In [9]:
#Get the before class activities and prepare and output a markdown file. 
schedule_bc=schedule.merge(content['Before Class'], left_on='Session', right_on='Session', how='left')
schedule_bc= schedule_bc.loc[schedule_bc['Content'].notnull(),['Week', 'Session', 'Date', 'Content']]
schedule_bc=bd.pandas_to_md(schedule_bc, content_path / 'preparation.md', 'Before Class', \
                             include = ['Week', 'Session', 'Date', 'Content'], header=cf['bc_class_header'])
#schedule=schedule.merge(content['Assignment'], left_on='Session', right_on='Session', how='left')



Converting datetime to 
Outputting file: ../site/content/preparation.md


In [12]:
schedule

,Week,Session,Date,Day,Topic,Summary,Publish,Type
0,1,1,08/30,Mon,Course Overview & Introduction to the Data Sci...,In this class we will simply be providing a hi...,1,Markdown
1,1,2,09/02,Thu,Python Basics,This lecture discusses the general strategic i...,1,Markdown
2,2,NaN,09/06,Mon,Labor day – no class,NaN,0,NaN
3,2,3,09/07,Tue,"Python Basics (First in Person Class, Tuesday...",This lecture discusses the general strategic i...,0,NaN
4,2,4,09/09,Thu,"Python conditionals, loops, functions, aggrega...",More operationalization of Python basics as th...,0,NaN
5,3,5,09/13,Mon,"Python conditionals, loops, functions, aggrega...",More operationalization of Python basics as th...,0,NaN
6,3,6,09/16,Thu,"Python visualization, data manipulation , and ...","Introduction to visualiation, APIs, web scrap...",0,NaN
7,4,7,09/20,Mon,"Python visualization, data manipulation , and ...","Introduction to visualiation, APIs, web scrap...",0,NaN
8,4,8,09/23,Thu,Overview of Modeling,We examine the basics of classess of supervise...,0,NaN
9,5,9,09/27,Mon,Overview of Modeling,We examine the basics of classess of supervise...,0,NaN


#Get the assignments and prepare and output a markdown file. 
assignments_new = schedule.loc[schedule['Assignment'].notnull(),['Week', 'Session', 'Date', 'Assignment', 'Due']]
if len(assignments_new)>0:
    assignments_new=bd.pandas_to_md(assignments_new, content_path / 'assignments.md', 'Assignments', \
                            include = ['Week', 'Session', 'Date', 'Assignment', 'Due'],header=cf['assignments_header'])

In [11]:
#Output the schedule to markdown.
schedule=bd.pandas_to_md(schedule, content_path / 'schedule.md', 'Schedule', \
                             include = ['Week', 'Session', 'Date', 'Day', 'Topic', 'Summary'],header=cf['schedule_header'])

Converting datetime to 


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
#Generate Session Files
#generate_sessions(config, toc, toc_part, schedule, path, content, keys):
bd.generate_sessions(config=config, toc=toc, schedule=schedule, path=class_path / 'sessions', content=content, keys=['Before Class', 'In Class'])

In [ ]:
#Update the sessions to the yaml file.  Other updates to notebooks need to be done manually.
#bd.update_yaml_file(class_path / '_toc.yml', toc)

In [ ]:
#TBD Make it so that notebooks will show up in _toc.yml. 